In [13]:
from turtle import window_height
from sklearn.preprocessing import LabelEncoder
from keras.applications import VGG19
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array, load_img
from keras import Sequential
from keras.layers import Conv2D, Flatten, Dense
from imutils import paths
import numpy as np
import pickle
import os
import argparse

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
parser = argparse.ArgumentParser()
parser.add_argument('-f --root', type=str, default="/content/drive/MyDrive/Kampus/ML/UAS/kaggle/", help='Root directory of data')
args = parser.parse_args()

In [16]:
# load the network and initialize the label encoder
print("[INFO] loading network...")
model = VGG19(weights="imagenet", include_top=False)
le = None

[INFO] loading network...


In [17]:
split = "mammals"
root_path = "/content/drive/MyDrive/Kampus/ML/UAS/kaggle/"
print("[INFO] processing '{} split'...".format(split))
p = os.path.sep.join([root_path, split])
imagePaths = list(paths.list_images(p))

[INFO] processing 'mammals split'...


In [18]:
# randomly shuffle the image paths and then extract the class
# labels from the file paths
# random.shuffle(imagePaths)
labels = [p.split(os.path.sep)[-2] for p in imagePaths]

In [19]:
# if the label encoder is None, create it
if le is None:
	le = LabelEncoder()
	le.fit(labels)

In [20]:
# open the output CSV file for writing
csvPath = os.path.sep.join(["/content/drive/MyDrive/Kampus/ML/UAS/VGG19_output",
	"{}.csv".format(split)])
csv = open(csvPath, "w")

In [21]:
# loop over the images in batches
for (b, i) in enumerate(range(0, len(imagePaths), 32)):
	# extract the batch of images and labels, then initialize the
	# list of actual images that will be passed through the network
	# for feature extraction
	print("[INFO] processing batch {}/{}".format(b + 1,
		int(np.ceil(len(imagePaths) / float(32)))))
	batchPaths = imagePaths[i:i + 32]
	batchLabels = le.transform(labels[i:i + 32])
	batchImages = []

	# loop over the images and labels in the current batch
	for imagePath in batchPaths:
		# load the input image using the Keras helper utility
		# while ensuring the image is resized to 224x224 pixels
		image = load_img(imagePath, target_size=(224, 224))
		image = img_to_array(image)

		# preprocess the image by (1) expanding the dimensions and
		# (2) subtracting the mean RGB pixel intensity from the
		# ImageNet dataset
		image = np.expand_dims(image, axis=0)
		image = imagenet_utils.preprocess_input(image)

		# add the image to the batch
		batchImages.append(image)

	# pass the images through the network and use the outputs as
	# our actual features, then reshape the features into a
	# flattened volume
	batchImages = np.vstack(batchImages)
	features = model.predict(batchImages, batch_size=32)
	features = features.reshape((features.shape[0], -1))



	# loop over the class labels and extracted features
	for (label, vec) in zip(batchLabels, features):
		# construct a row that exists of the class label and
		# extracted features
		vec = ",".join([str(v) for v in vec])
		csv.write("{},{}\n".format(vec, label))

# close the CSV file
csv.close()

# serialize the label encoder to disk
f = open("/content/drive/MyDrive/Kampus/ML/UAS/VGG19_output/le.cpickle", "wb")
f.write(pickle.dumps(le))
f.close()

[INFO] processing batch 1/430
1/1 [==============================] - 27s 27s/step
[INFO] processing batch 2/430
1/1 [==============================] - 24s 24s/step
[INFO] processing batch 3/430
1/1 [==============================] - 23s 23s/step
[INFO] processing batch 4/430
1/1 [==============================] - 23s 23s/step
[INFO] processing batch 5/430
1/1 [==============================] - 26s 26s/step
[INFO] processing batch 6/430
1/1 [==============================] - 25s 25s/step
[INFO] processing batch 7/430
1/1 [==============================] - 25s 25s/step
[INFO] processing batch 8/430
1/1 [==============================] - 25s 25s/step
[INFO] processing batch 9/430
1/1 [==============================] - 24s 24s/step
[INFO] processing batch 10/430
1/1 [==============================] - 27s 27s/step
[INFO] processing batch 11/430
1/1 [==============================] - 24s 24s/step
[INFO] processing batch 12/430
1/1 [==============================] - 24s 24s/step
[INFO] proces

In [22]:
print("Extracting features is done!")

Extracting features is done!
